In [55]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import joblib

# Importing new Datasets from Huggingface

In [27]:
df_v2_2 = pd.read_parquet("hf://datasets/kinisi/gym_kr2-PickYCB-v1-fixed-depth_generated_v2.2/data/chunk-000/episode_000000.parquet")
df_fake_v2_2 = pd.read_parquet("hf://datasets/kinisi/gym_kr2-PickYCB-v1-fixed-depth_fake_depth_generated_v2.2/data/chunk-000/episode_000000.parquet")
df_v2_3 = pd.read_parquet("hf://datasets/kinisi/gym_kr2-PickYCB-v1-fixed-depth_generated_v2.3/data/chunk-000/episode_000000.parquet")
df_fake_v2_3 = pd.read_parquet("hf://datasets/kinisi/gym_kr2-PickYCB-v1-fixed-depth_fake_depth_generated_v2.3/data/chunk-000/episode_000000.parquet")


# Old data 
### Generated V2.1 and Fake V2.1

In [31]:
state_old = pd.read_csv('../data/observation_data')
action_old = pd.read_csv('../data/action_data')

In [28]:
state_new = pd.concat([df_v2_2['observation.state'], df_fake_v2_2['observation.state'], df_v2_3['observation.state'], df_fake_v2_3['observation.state']], ignore_index=True)
action_new = pd.concat([df_v2_2['action'], df_fake_v2_2['action'], df_v2_3['action'], df_fake_v2_3['action']], ignore_index=True)

In [44]:
state_df = pd.DataFrame(state_new.tolist())
state_df.columns = [f'state_{i}' for i in range(state_df.shape[1])]

action_df = pd.DataFrame(action_new.tolist())
action_df.columns = [f'state_{i}' for i in range(action_df.shape[1])]

# Spliting Data into train and test set

In [45]:
X = pd.concat([state_old, state_df], ignore_index=True)
y = pd.concat([action_old, action_df], ignore_index=True)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

In [47]:
X_train.shape

(6699, 31)

In [48]:
y_test.shape

(2871, 21)

# Scaling Datasets

In [49]:
scaler_X = StandardScaler()
X_train_s = scaler_X.fit_transform(X_train)
X_test_s = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_s = scaler_y.fit_transform(y_train)
y_test_s = scaler_y.transform(y_test)

# SKlearn MLP regressor model

In [50]:
mlp = MLPRegressor(max_iter=1000, activation='relu', hidden_layer_sizes=(100,), random_state=12)
mlp.fit(X_train_s, y_train_s)
y_pred = mlp.predict(X_test_s)


# Calculating R2 score

In [53]:
score = r2_score(y_test_s, y_pred)
print(score)

0.8080351833701049


# Dataset creation

In [54]:
# Creating new dataset combined of v2.1, v2.2 and v2.3 both generated and fake_generated 
X.to_csv('../data/observation_new', index=False)
y.to_csv('../data/action_new', index=False)

# Saving model and scaler 

In [56]:
joblib.dump(scaler_X, '../models/scaler_X.pkl')
joblib.dump(scaler_y, '../models/scaler_y.pkl')
joblib.dump(mlp, '../models/mlp.pkl')

['../models/mlp.pkl']